In [1]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00


In [2]:
!pip install pymupdf
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [5]:
import os
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 4.1 MB/s eta 0:00:00


**Appropriateness and justification of methodologies**

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import pymupdf
class content_eval_agent:
    def __init__(self, model, api_key, content):
        self.llm=ChatGroq(model=model, api_key=api_key)
        self.system_prompt= '''You are an Appropriateness analyzer of research papers.
        Analyze the appropriateness and justification of the methodologies used in this research paper.
         Discuss whether the methods align with the research objectives, and evaluate their strengths and weaknesses in addressing the research questions.
         Provide an assessment of the validity, reliability, and generalizability of the chosen methodologies.
         Include specific examples from the paper to support your analysis.
         Specifically:
         1. Mention if the chosen methods directly support the research goals and questions.
         2. Whether the paper has Provided evidence from past research or pilot studies that supports the selection of the methodology.
         3. Whether the methods are practically implemented within available resources.
         4. Whether the paper used a lot of terms and concepts are used throughout without proper explanation.

         Positive score is given if the above first 3 points are supported, last one is negative point.Score the paper out of 10. '''

        self.eval_prompt = ChatPromptTemplate.from_messages(
            [

                    ("system", self.system_prompt),


                    ( "user", "This is the content of the research paper:\n\n {content}.\n\n Please evaluate it and rate the overall quality of paper out of 10.",)
            ,
            ]
        )
        self.content_agent=self.eval_prompt | self.llm | StrOutputParser()

    def evaluate_content(self, content: str) -> str:
        evaluation = self.content_agent.invoke({"content": content})
        return evaluation



from math import ceil

def split_text(text, max_tokens=6000):
    """
    Splits text into smaller chunks to meet token limit.

    Args:
        text (str): The text to split.
        max_tokens (int): Maximum tokens allowed in each chunk.

    Returns:
        list: List of text chunks.
    """
    words = text.split()
    chunk_size = max_tokens
    chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def extract_and_chunk_paper(pdf_path, max_tokens=2000):
    """
    Extracts headings and content from a PDF and ensures they fit within token limits.

    Args:
        pdf_path (str): Path to the PDF file.
        max_tokens (int): Maximum tokens per chunk.

    Returns:
        list: List of dictionaries with headings and corresponding content chunks.
    """
    doc = pymupdf.open(pdf_path)
    headings_content = {}
    current_heading = "General"
    content_buffer = ""

    for page in doc:
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    text = " ".join(span["text"] for span in line["spans"]).strip()
                    font_size = line["spans"][0]["size"]

                    if font_size > 12:
                        if current_heading and content_buffer:
                            headings_content[current_heading] = content_buffer.strip()
                        current_heading = text
                        content_buffer = ""
                    else:
                        content_buffer += " " + text

    if current_heading and content_buffer:
        headings_content[current_heading] = content_buffer.strip()

    doc.close()


    chunked_output = []
    for heading, content in headings_content.items():
        chunks = split_text(content, max_tokens=max_tokens)
        for idx, chunk in enumerate(chunks):
            chunked_output.append({
                "heading": f"{heading} (Part {idx + 1})" if len(chunks) > 1 else heading,
                "content": chunk
            })
            break
        break

    return chunked_output


if __name__ == "__main__":
    pdf_path = "/content/20267_Robust_Preference_Optimi_removed.pdf"
    headings_content = extract_and_chunk_paper(pdf_path)


    # for heading, content in headings_content.items():
    #     print(f"== {heading} ==")
    #     print(content if content else "No content found.")
    #     print("\n")

    model="llama3-70b-8192"
    api_key="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"
    content_evaluator = content_eval_agent(model, api_key, headings_content)
    evaluation = content_evaluator.evaluate_content(headings_content)
    print(evaluation)




After analyzing the research paper, I provide my assessment of the appropriateness and justification of the methodologies used.

**Alignment with Research Objectives:**
The paper's methodology directly supports the research goals and questions. The authors propose a method that merges the best of both worlds: an efficient reward model distillation algorithm that operates effectively in the offline setting, makes minimal assumptions about the true, optimal reward, and demonstrates greater robustness to the specific distribution of prompt/response data used for policy alignment.

**Evidence from Past Research or Pilot Studies:**
The paper provides evidence from past research, citing various studies and techniques, such as Direct Preference Optimization (DPO), Bradley-Terry models, and knowledge distillation techniques. These references support the selection of the methodology and provide a solid foundation for the proposed approach.

**Practical Implementation within Available Resources:

**Content Evaluation agent**

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
import pymupdf
class content_eval_agent:
    def __init__(self, model, api_key, content):
        self.llm=ChatGroq(model=model, api_key=api_key)
        self.system_prompt= "You are a content evaluation assistant for research papers. Your task is to assess whether the "
        "provided research paper includes all critical sections required for publication and if the content "
        "in each section is sufficiently detailed and relevant.\n\n"
        "Specifically:\n"
        "1. Ensure the paper includes the following sections: Abstract, Introduction, Methodology, Results, "
        "Discussion, and Conclusion.\n"
        "2. Highlight missing sections, if any.\n"
        "3. Identify sections with insufficient or irrelevant details.\n"
        "4. Provide actionable feedback on how the paper can be improved.\n"
        "5. Ensure your response is structured as a checklist followed by recommendations.\n\n"
        "6. Give a rating out of 10 for the overall quality of the paper.\n\n"
        "Return your evaluation in the following format:\n"
        "- Section: [Present/Missing/Insufficient]\n"
        "- Feedback: [Detailed feedback for improvement or acknowledgment if the section is good]\n"
        "- Rating: [Rating out of 10]\n\n"

        self.eval_prompt = ChatPromptTemplate.from_messages(
            [

                    ("system", self.system_prompt),


                    ( "user", "This is the content of the research paper:\n\n {content}.\n\n Please evaluate it and rate the overall quality of paper out of 10.",)
            ,
            ]
        )
        self.content_agent=self.eval_prompt | self.llm | StrOutputParser()

    def evaluate_content(self, content: str) -> str:
        evaluation = self.content_agent.invoke({"content": content})
        return evaluation



from math import ceil

def split_text(text, max_tokens=6000):
    """
    Splits text into smaller chunks to meet token limit.

    Args:
        text (str): The text to split.
        max_tokens (int): Maximum tokens allowed in each chunk.

    Returns:
        list: List of text chunks.
    """
    words = text.split()
    chunk_size = max_tokens
    chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def extract_and_chunk_paper(pdf_path, max_tokens=2000):
    """
    Extracts headings and content from a PDF and ensures they fit within token limits.

    Args:
        pdf_path (str): Path to the PDF file.
        max_tokens (int): Maximum tokens per chunk.

    Returns:
        list: List of dictionaries with headings and corresponding content chunks.
    """
    doc = pymupdf.open(pdf_path)
    headings_content = {}
    current_heading = "General"
    content_buffer = ""

    for page in doc:
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    text = " ".join(span["text"] for span in line["spans"]).strip()
                    font_size = line["spans"][0]["size"]

                    if font_size > 12:
                        if current_heading and content_buffer:
                            headings_content[current_heading] = content_buffer.strip()
                        current_heading = text
                        content_buffer = ""
                    else:
                        content_buffer += " " + text

    if current_heading and content_buffer:
        headings_content[current_heading] = content_buffer.strip()

    doc.close()


    chunked_output = []
    for heading, content in headings_content.items():
        chunks = split_text(content, max_tokens=max_tokens)
        for idx, chunk in enumerate(chunks):
            chunked_output.append({
                "heading": f"{heading} (Part {idx + 1})" if len(chunks) > 1 else heading,
                "content": chunk
            })
            break
        break

    return chunked_output


if __name__ == "__main__":
    pdf_path = "/content/20926_AlignedCut_Visual_Concep_removed.pdf"
    headings_content = extract_and_chunk_paper(pdf_path)


    # for heading, content in headings_content.items():
    #     print(f"== {heading} ==")
    #     print(content if content else "No content found.")
    #     print("\n")

    model="llama3-70b-8192"
    api_key="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"
    content_evaluator = content_eval_agent(model, api_key, headings_content)
    evaluation = content_evaluator.evaluate_content(headings_content)
    print(evaluation)




I'll evaluate the paper based on various aspects, including content, structure, language, and overall quality.

**Content (8/10)**

The paper presents an innovative approach to creating a universal feature space for deep neural networks using brain voxel response prediction as the training objective. The authors demonstrate that channel features across different networks can be aligned and clustered to form meaningful visual concepts. The paper also explores the relationship between brain regions and channel activations, which is an interesting and novel aspect of the research.

The content is well-organized, and the authors provide a clear introduction to the problem, followed by a detailed explanation of their methodology and results. The paper includes several figures and equations that help to illustrate the concepts and methods.

However, the paper could benefit from more context and background information on the relevant topics, such as brain-computer interfaces, neural networks,